In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F 
from pyspark.sql.types import *
from delta import *

In [2]:
builder = SparkSession.builder.appName("olist_demo") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .config("spark.sql.repl.eagerEval.enabled", "true") \
    .config("spark.executor.memory", "2g")\
    .config("spark.driver.memory", "4g")\
    .config("spark.driver.memory", "4g")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/home/fabio/Repos/olist-pyspark-elt-demo/.venv/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/fabio/.ivy2/cache
The jars for the packages stored in: /home/fabio/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-edfa0177-ff9e-4e32-9219-d665285533ff;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.2.1 in central
	found io.delta#delta-storage;3.2.1 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 118ms :: artifacts dl 5ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.2.1 from central in [default]
	io.delta#delta-storage;3.2.1 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   

## Config

In [3]:
environment = '' # dev;uat;prd

In [ ]:
base_path = '../../data'  # This will be overwritten by Papermill

In [4]:
path = {
    'stg': {'orders': f'{base_path}/stage/olist_orders_dataset.csv'},
    'snk': f'{base_path}/bronze/orders'
}

In [5]:
options = {
    "inferSchema": "True",
    "delimiter": ",",
    "header": "True"
}

## Load

In [6]:
df = spark.read.options(**options).format('csv').load(path['stg']['orders'])

In [7]:
df_snk = df.select(
    F.col("order_id").cast("string"),
    F.col("customer_id").cast("string"),
    F.col("order_status").cast("string"),
    F.col("order_purchase_timestamp").cast("timestamp"),
    F.col("order_approved_at").cast("timestamp"),
    F.col("order_delivered_carrier_date").cast("timestamp"),
    F.col("order_delivered_customer_date").cast("timestamp"),
    F.col("order_estimated_delivery_date").cast("timestamp")
)

In [8]:
df_snk = df_snk.withColumn("datetime_insert_brz", F.current_timestamp())

## Sink

In [9]:
df_snk.write.format('delta')\
.option("overwriteSchema", "true")\
.mode(saveMode='overwrite')\
.save(path['snk'])

24/10/03 11:28:21 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
